<a href="https://colab.research.google.com/github/s2t2/ml-music/blob/main/notebooks/youtube/Training_Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Development Plan

  1. Login with google, access and mount the drive
  2. Pick a given number of mfccs (2, 3, 12, 20). Or maybe start with 20 (to match GTZAN).
  3. Get all the artists. for each:
    + Get all the videos. for each:
      + Get all the tracks. for each:
         + Read the MFCCs CSV file
         + Take the mean and variance of each mfcc (i.e. summaries) and add to the benchmark model training file
         + TODO: Append the mfccs themselves to a deep learning training file


  4. Save one CSV file of all tracks summary data, "tracks_mfcc_12_summary.csv", to the "youtube_channels" directory.


  TODO: 5. Save one CSV file of all tracks data, "tracks_mfcc_12.csv", to the "youtube_channels" directory.

## Setup

In [71]:
#%%capture

# https://pytube.io/en/latest/index.html
# https://librosa.org/doc/main/generated/librosa.feature.mfcc.html
# https://pysoundfile.readthedocs.io/en/latest/#module-soundfile

#!pip install pytube librosa soundfile

In [72]:
import warnings

warnings.filterwarnings("ignore")

In [73]:
#!pip list

### Google Drive

In [74]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
import os

# you might need to update the path below, or create a shortcut to the path below
DATASET_PATH = '/content/drive/MyDrive/Research/DS Research Shared 2023/data/ml_music_2023'

print(DATASET_PATH)
assert os.path.isdir(DATASET_PATH)

/content/drive/MyDrive/Research/DS Research Shared 2023/data/ml_music_2023


In [76]:
CHANNELS_DIRPATH = os.path.join(DATASET_PATH, "youtube_channels")
channel_names = os.listdir(CHANNELS_DIRPATH)
print(channel_names)

['maggie_rogers', 'john_mayer', 'chris_stapleton']


## Compiling Audio Features (MFCCs)

In [77]:
from pandas import read_csv, DataFrame, Series

#NUMS_MFCC = [2, 3, 12, 20]
N_MFCC = 20

rows = []

for channel_name in channel_names:
    print(channel_name.upper())

    channel_dirpath = os.path.join(CHANNELS_DIRPATH, channel_name)
    video_ids = os.listdir(channel_dirpath)
    print(len(video_ids), "VIDEOS...")
    
    for video_id in video_ids:
        #print(video_id)
        video_dirpath = os.path.join(channel_dirpath, video_id)
        mfcc_dirpath = os.path.join(video_dirpath, f"mfcc_{N_MFCC}")

        for track_filename in os.listdir(mfcc_dirpath):
            row = {
                "channel_name": channel_name,
                "video_id": video_id,
                "track_name": track_filename.split(".csv")[0]
            }

            track_filepath = os.path.join(mfcc_dirpath, track_filename)    
            mfcc_df = read_csv(track_filepath)
            mfcc_df.drop(columns=["Unnamed: 0"], inplace=True)
            
            for mfcc_colname in mfcc_df.columns.tolist():
                track_mfcc = mfcc_df[mfcc_colname]
                row[f"{mfcc_colname}_mean"] = track_mfcc.mean()
                row[f"{mfcc_colname}_var"] = track_mfcc.std()

            rows.append(row)

all_tracks_df = DataFrame(rows)
all_tracks_df

MAGGIE_ROGERS
9 VIDEOS...
JOHN_MAYER
10 VIDEOS...
CHRIS_STAPLETON
12 VIDEOS...


,channel_name,video_id,track_name,mfcc_1_mean,mfcc_1_var,mfcc_2_mean,mfcc_2_var,mfcc_3_mean,mfcc_3_var,mfcc_4_mean,mfcc_4_var,mfcc_5_mean,mfcc_5_var,mfcc_6_mean,mfcc_6_var,mfcc_7_mean,mfcc_7_var,mfcc_8_mean,mfcc_8_var,mfcc_9_mean,mfcc_9_var,mfcc_10_mean,mfcc_10_var,mfcc_11_mean,mfcc_11_var,mfcc_12_mean,mfcc_12_var,mfcc_13_mean,mfcc_13_var,mfcc_14_mean,mfcc_14_var,mfcc_15_mean,mfcc_15_var,mfcc_16_mean,mfcc_16_var,mfcc_17_mean,mfcc_17_var,mfcc_18_mean,mfcc_18_var,mfcc_19_mean,mfcc_19_var,mfcc_20_mean,mfcc_20_var
0,maggie_rogers,0dzZXpf7sSQ,track_1,-247.604264,126.564314,94.436743,44.407494,-17.821985,39.812022,5.147592,21.637402,-5.117160,19.775668,-2.886482,15.934825,-9.404755,14.546043,2.010384,12.245708,-3.486708,12.681550,-2.080226,12.014526,-11.123175,9.881194,-8.429318,10.043300,-11.683237,8.611424,-6.062387,10.251812,-9.810182,10.150266,-9.767659,9.709967,-15.159890,10.092836,-5.886746,8.073567,-3.599840,7.643674,-0.015861,7.409028
1,maggie_rogers,0dzZXpf7sSQ,track_2,-283.419875,68.689398,108.620859,45.736677,22.183784,23.642963,36.270842,18.142406,4.363370,13.825269,-10.048605,17.788155,6.312012,16.853004,-20.865720,16.432897,-12.531060,11.710459,4.237882,17.082234,-28.417574,14.621805,5.096091,13.747154,-28.501671,14.691406,0.945887,13.145683,-11.319688,12.450618,-20.554510,14.498205,-6.835958,12.063171,-8.068021,14.051340,-3.437384,10.086706,8.037604,14.541075
2,maggie_rogers,0dzZXpf7sSQ,track_3,-191.882983,87.372699,118.984447,32.211981,8.047093,18.469054,16.937100,13.607642,-1.512961,9.572726,-2.478823,13.878476,3.151490,9.802588,-3.808886,11.449715,-7.606141,9.706977,-0.659397,10.306092,-17.404371,10.002556,2.601389,10.611561,-13.998166,16.333500,4.186738,11.095053,-7.014209,8.475307,-6.896051,12.169556,-4.585148,6.950206,-3.239831,7.794687,-4.770292,8.486704,-1.037769,8.149938
3,maggie_rogers,0dzZXpf7sSQ,track_4,-182.191475,71.259583,74.172125,38.583971,0.747170,18.817206,21.487847,14.833436,-4.348487,11.652041,-1.332470,18.995348,-1.436782,11.363993,-5.561523,17.340706,-8.219880,12.261442,-0.034450,9.967833,-17.926652,12.873735,-2.263728,8.872252,-12.753958,14.671951,1.837090,11.858167,-6.655334,11.009060,-5.200492,13.869197,-5.368410,10.311301,-1.005449,8.997953,-2.508238,9.274194,5.581843,12.752089
4,maggie_rogers,0dzZXpf7sSQ,track_5,-204.492367,68.350796,101.394189,46.044563,21.449712,21.587820,38.283024,15.049941,10.165238,14.806656,2.464657,15.246444,3.999687,12.650244,-1.221194,10.597701,-4.007767,9.427915,1.995180,10.032767,-14.467728,10.961002,3.488676,10.873569,-17.618445,11.235162,-1.922295,8.974619,-10.732721,8.648910,-7.574725,10.880968,-6.204093,9.224697,-2.311896,8.471938,-2.994891,7.776321,5.986408,10.474859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,chris_stapleton,l6_w3887Rwo,track_5,-126.295066,76.964783,105.274319,32.933817,-2.122840,20.203390,37.958635,15.411133,4.583544,14.222600,21.271266,10.963622,3.376333,11.453415,2.706425,9.797716,-0.148747,9.176137,4.388386,10.230980,-1.104775,9.203958,6.030791,7.822974,-2.653412,8.722891,0.340779,7.438175,-3.880533,7.799143,-2.008210,8.618912,-3.950875,9.735785,2.288542,9.793921,-6.366817,7.404871,-1.547669,10.210957
239,chris_stapleton,l6_w3887Rwo,track_6,-122.054477,69.334728,111.313190,28.755073,-4.131346,20.107382,31.233111,14.249997,1.186262,18.077495,21.589101,9.316963,4.229436,11.662096,4.240373,7.963354,-4.274452,9.073208,5.366055,7.636164,-1.869967,8.523571,7.269486,8.814108,-1.092924,8.809050,1.191934,7.658668,-5.184975,8.018576,-3.013207,6.449025,-8.324395,7.403035,-2.395323,8.365698,-5.696421,8.885964,1.138039,9.225098
240,chris_stapleton,l6_w3887Rwo,track_7,-125.434740,66.307994,109.874518,22.530859,-0.937652,16.624893,32.527456,9.195076,11.426024,9.682593,17.477038,8.411168,-1.131768,8.102473,5.935303,7.577918,-5.133981,6.512568,4.536335,7.513250,-4.167491,8.196160,4.298700,7.706451,-0.664826,7.921739,3.6

In [84]:
csv_filepath = os.path.join(DATASET_PATH, "30s_features_mfcc_20.csv")

all_tracks_df.to_csv(csv_filepath)

## Scratch Work

In [79]:
from pandas import read_csv


channel_name = "maggie_rogers"
video_id = "0dzZXpf7sSQ"
track_filename = "track_1.csv"
track_filepath = f"/content/drive/MyDrive/Research/DS Research Shared 2023/data/ml_music_2023/youtube_channels/{channel_name}/{video_id}/mfcc_20/{track_filename}"

mfcc_df = read_csv(track_filepath)
mfcc_df.drop(columns=["Unnamed: 0"], inplace=True)
print(mfcc_df.shape)
mfcc_df.head()

(1292, 20)


,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20
0,-572.910829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-572.910829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-572.910829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-572.910829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-572.910829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
mfcc_df["mfcc_1"].mean()

-247.60426353482953

In [81]:
#@title
#from pandas import DataFrame
#
## there is probably a way to manipulate the df but let's loop for now
#
#
#
#results = []
#for mfcc_colname in mfcc_df.columns.tolist():
#    mfcc = mfcc_df[mfcc_colname]
#
#    result = {
#        "channel_name": channel_name,
#        "video_id": video_id,
#        "track_name": track_filename.split(".csv")[0],
#        f"{mfcc_colname}_mean": mfcc.mean(), 
#        #f"{mfcc_colname}_std": mfcc.std(), 
#        f"{mfcc_colname}_var": mfcc.var(), 
#    }
#    print(result)
#    results.append(result)
#
#
#mfcc_summary_df = DataFrame(results)
#print(mfcc_summary_df.shape)
#mfcc_summary_df.head()
##  4. Save one CSV file of all tracks summary data, "tracks_mfcc_12_summary.csv", to the "youtube_channels" directory.

In [82]:
from pandas import DataFrame, Series

# we want a single dictionary with all the mfcc summaries

row = {"channel_name": channel_name,
        "video_id": video_id,
        "track_name": track_filename.split(".csv")[0]}
        
for mfcc_colname in mfcc_df.columns.tolist():
    mfcc = mfcc_df[mfcc_colname]
    row[f"{mfcc_colname}_mean"] = mfcc.mean()
    row[f"{mfcc_colname}_var"] = mfcc.std()

row

{'channel_name': 'maggie_rogers',
 'video_id': '0dzZXpf7sSQ',
 'track_name': 'track_1',
 'mfcc_1_mean': -247.60426353482953,
 'mfcc_1_var': 126.56431387454751,
 'mfcc_2_mean': 94.43674305967087,
 'mfcc_2_var': 44.407494167772946,
 'mfcc_3_mean': -17.821985329535504,
 'mfcc_3_var': 39.81202174948487,
 'mfcc_4_mean': 5.147591517832438,
 'mfcc_4_var': 21.63740240230002,
 'mfcc_5_mean': -5.1171596284261796,
 'mfcc_5_var': 19.77566752579058,
 'mfcc_6_mean': -2.886482119649188,
 'mfcc_6_var': 15.934824627993338,
 'mfcc_7_mean': -9.404755079522932,
 'mfcc_7_var': 14.546042574637253,
 'mfcc_8_mean': 2.010383759948643,
 'mfcc_8_var': 12.245707525348202,
 'mfcc_9_mean': -3.4867082601556256,
 'mfcc_9_var': 12.681549631715813,
 'mfcc_10_mean': -2.0802263753650476,
 'mfcc_10_var': 12.014525832536421,
 'mfcc_11_mean': -11.123175448600486,
 'mfcc_11_var': 9.881194154921852,
 'mfcc_12_mean': -8.429318324339611,
 'mfcc_12_var': 10.04330010274523,
 'mfcc_13_mean': -11.683236931640073,
 'mfcc_13_var': 8.

In [83]:


#mfcc_dirpath = os.path.join(CHANNELS_DIRPATH, "maggie_rogers", "0dzZXpf7sSQ", "mfcc_20")
#
#for track_filename in os.listdir(mfcc_dirpath):
#    print(track_filename)
#    track_filepath = os.path.join(mfcc_dirpath, track_filename)
#    print(track_filepath)
#    track_df = read_csv(track_filepath)
#    print(track_df.head())